# Import the required libraries

In [141]:
import numpy as np
import pandas as pd
import yfinance as yf

# Define a buy function

In [142]:
def buy(entry, amount = None, share_unit = None, ticker = ticker):

    global balance
    global share
    
    time, column = entry
    
    price = column['Close']

    if share_unit is None: 
        share_unit = amount/price
    
    if share_unit >= 1.0:   # Minimum transaction size
        balance -= share_unit*price 
        share += share_unit
   
    #print(f'{time}:  Buy {round(share_unit, 2)} {ticker} shares for ${round(price, 2)}.')


# Define a sell function

In [143]:
def sell(entry, amount = None, share_unit = None, ticker = ticker):

    global balance
    global share
    
    time, column = entry
    
    price = column['Close']

    if share_unit is None: 
        share_unit = amount/price

    if share_unit >= 1.0:   # Minimum transaction size  
        balance += share_unit*price
        share -= share_unit  

    #print(f'{time}:  Sell {round(share_unit, 2)} {ticker} shares for ${round(price, 2)}.')

# Define a function to execute trades based on the generated trading signals 

In [144]:
def execute_trade(position, signal, entry):
    
    time, column = entry
    
    if signal == +1:
        
        if position == 0:
            buy(entry, amount = balance)
        
        elif position == -1: # Close the short position first to establish a neutral position
            buy(entry, share_unit = -share)
            buy(entry, amount = balance)
        
    elif signal == -1: 
        
        if position == 0:
            sell(entry, amount = balance)
        
        elif position == +1: # Close the long position first to establish a neutral position
            sell(entry, share_unit = share)
            sell(entry, amount = balance)
    
    elif signal == 0:
        
        if position == +1:
            sell(entry, share_unit = share)
        
        elif position == -1:
            buy(entry, share_unit = -share)
    

# Functions of the Performance Metrics 

# Total & Annual Returns

In [145]:
def returns(data, total_trading_days):
    data['Total_Returns'] = (data['Strategy_Returns'] + 1).cumprod()
    total_returns = data.iloc[-1]['Total_Returns']*100
    annual_returns = (data.iloc[-1]['Total_Returns']**(252/total_trading_days) - 1)*100
        
    #print(f'Total Returns: {round(total_returns, 2)}%')
    #print(f'Annual Returns: {round(annual_returns, 2)}%')
    return round(total_returns, 2), round(annual_returns, 2)

# Annual Volatility

In [146]:
def annual_volatility(data):
    annual_vol = (data['Strategy_Returns'].std())*(252**0.5)*100
    #print(f'Annual Volatility: {round(annual_volatility, 2)}%')
    return round(annual_vol, 2)

# Sharpe Ratio

In [147]:
def sharpe_ratio(data, rf):
    sharpe_rat = (252**0.5)*(data['Strategy_Returns'].mean() - rf)/(data['Strategy_Returns'].std())
    #print(f'Sharpe Ratio: {round(sharpe_ratio, 2)}')
    return round(sharpe_rat, 2)

# Sortino Ratio

In [148]:
def sortino_ratio(data, rf):
    negative_strat_returns = data[data['Strategy_Returns'] < 0]
    sortino_rat = (252**0.5)*(data['Strategy_Returns'].mean() - rf)/(negative_strat_returns['Strategy_Returns'].std())
    #print(f'Sortino Ratio: {round(sortino_ratio, 2)}')
    return round(sortino_rat, 2)

# Maximum Drawdown

In [149]:
def max_drawdown(data):
    running_max = data['Total_Returns'].cummax() 
    running_max[running_max < 1] = 1
    drawdown = (data['Total_Returns'])/running_max - 1
    max_dd = drawdown.min()*100
    #print(f'Maximum Drawdown: {round(max_drawdown, 2)}%')
    return round(max_dd, 2)

# Create a backtester function 

In [150]:
def backtester(position, share, rf, start_date, end_date, ticker):
    
    # Download the historical price data
    data = yf.download(ticker, start = start_date, end = end_date)

    # Compute the Double Bollinger Bands  
    data['SMA'] = data['Close'].rolling(20).mean()
    data['Std'] = data['Close'].rolling(20).std()

    data['A1'] = data['SMA'] + 2*data['Std']
    data['A2'] = data['SMA'] - 2*data['Std']

    data['B1'] = data['SMA'] + 1*data['Std']
    data['B2'] = data['SMA'] - 1*data['Std']

    data.drop(['Open', 'High', 'Low', 'Volume'], axis = 1, inplace = True, errors = 'ignore')
    data = data.dropna(axis = 0)

    # Generate the trading signals 
    data['Signal'] = np.where(data['Close'] > data['B1'], +1, 0)
    data['Signal'] = np.where(data['Close'] < data['B2'], -1, data['Signal'])

    # Calculate the daily returns based on the closing prices
    data['Returns'] = data['Close'].pct_change().fillna(0)

    # Calculate strategy returns
    data['Strategy_Returns'] = data['Returns'] * data['Signal'].shift(1).fillna(0)

    # Total number of trading days 
    total_trading_days = len(data)  

    # Iterate through each row of the stock dataframe
    for num, entry in enumerate(data.iterrows()):

        time, column = entry

        if num == data.shape[0] - 1: # Close all positions for the last trade
            signal = 0
        else:
            signal = column['Signal']

        execute_trade(position, signal, entry)

        position = signal # Update the latest position

    #print('\n')
    #print(f'Stock Ticker: {ticker}')
    #print(f'Final Capital Balance: ${round(balance, 2)}')
    
    # Compute the performance metrics 
    total_returns, annual_returns = returns(data, total_trading_days)
    annual_vol = annual_volatility(data)
    sharpe_rat = sharpe_ratio(data, rf)
    sortino_rat = sortino_ratio(data, rf)
    max_dd = max_drawdown(data) 

    return total_returns, annual_returns, annual_vol, sharpe_rat, sortino_rat, max_dd

In [151]:
# Magnificent 7 Tickers  
mag_7 = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']


# DF Columns 
total_return_list = []
annual_return_list = []
annual_volatility_list = [] 
sharpe_ratio_list = []
sortino_ratio_list = []
maximum_drawdown_list = []
final_cap_balance_list = []


# Loop through each stock ticker 
for ticker in mag_7:
    balance = 10000 # Initial capital balance
    position = 0 # Initial trading position
    share = 0 # Initial amount of shares owned
    rf = 0.01/252  # Assume an annual risk-free interest rate of 1%
    start_date = '2013-01-01'
    end_date = '2023-12-31'
    
    total_returns, annual_returns, annual_vol, sharpe_rat, sortino_rat, max_dd = backtester(position, share, rf, start_date, end_date, ticker = ticker)
    
    # Append values to columns
    final_cap_balance_list.append(round(balance, 2))
    total_return_list.append(total_returns)
    annual_return_list.append(annual_returns)
    annual_volatility_list.append(annual_vol) 
    sharpe_ratio_list.append(sharpe_rat)
    sortino_ratio_list.append(sortino_rat)
    maximum_drawdown_list.append(max_dd)
    

summary = {'Final Capital Balance ($)': final_cap_balance_list,
'Total Return (%)': total_return_list,
'Annual Return (%)': annual_return_list,
'Annual Volatility (%)': annual_volatility_list,
'Sharpe Ratio': sharpe_ratio_list,
'Sortino Ratio': sortino_ratio_list,
'Maximum Drawdown (%)': maximum_drawdown_list}

df = pd.DataFrame(summary, index = mag_7)

[*********************100%%**********************]  1 of 1 completed
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = np.where(data['Close'] > data['B1'], +1, 0)
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = np.where(data['Close'] < data['B2'], -1, data['Signal'])
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:24: SettingWithCopy

[*********************100%%**********************]  1 of 1 completed
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = np.where(data['Close'] > data['B1'], +1, 0)
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = np.where(data['Close'] < data['B2'], -1, data['Signal'])
C:\Users\kokwa\AppData\Local\Temp\ipykernel_14604\237251818.py:24: SettingWithCopy

In [165]:
df

,Final Capital Balance ($),Total Return (%),Annual Return (%),Annual Volatility (%),Sharpe Ratio,Sortino Ratio,Maximum Drawdown (%)
MSFT,3370.88,29.15,-10.69,19.59,-0.53,-0.50,-75.87
AAPL,20666.36,179.17,5.49,20.64,0.31,0.31,-34.47
NVDA,10957.01,82.19,-1.78,32.94,0.08,0.09,-57.12
AMZN,10536.67,90.92,-0.87,24.80,0.05,0.05,-47.30
GOOG,4556.64,39.49,-8.16,20.36,-0.37,-0.37,-65.85
META,3226.04,26.02,-11.61,29.34,-0.31,-0.30,-87.07
TSLA,334968.55,2351.55,33.57,45.08,0.84,0.90,-70.27
